# **Preventive Care and Health Screening System**

### **Problem Statement**

In modern healthcare, preventive care and health screenings are essential for early detection and effective management of chronic diseases. However, many patients miss their routine screenings due to a lack of timely reminders or awareness. This results in late diagnoses, higher healthcare costs, and poorer health outcomes.

Healthcare providers face challenges in identifying and reaching patients who are due or overdue for preventive screenings. They need a systematic, automated solution to manage patient data, track screening schedules, and provide timely notifications.

---

### **Core Problem Statement**

**"How can we build an automated and cost-effective system that identifies patients overdue for preventive care and health screenings, predicts their risk levels, and sends timely reminders to improve adherence and health outcomes?"**

---

### **Challenges Addressed**
1. **Identification of Overdue Patients**  
   Patients who miss their screenings are often not identified due to fragmented or outdated patient records.

2. **Risk Prediction**  
   Healthcare providers need to prioritize high-risk patients for follow-ups, but manual analysis is time-consuming and error-prone.

3. **Timely Notifications**  
   Many patients do not receive reminders about their preventive care, leading to missed opportunities for early intervention.

4. **Resource Constraints**  
   Many healthcare organizations have limited budgets and resources to develop sophisticated systems.

---

### **Goals of the Solution**
1. **Automate Screening Identification**  
   Develop a system to automatically flag patients overdue for screenings.

2. **Predict Risk Levels**  
   Use machine learning to predict the likelihood of complications based on patient history.

3. **Send Automated Notifications**  
   Integrate notification mechanisms (SMS, email) to remind patients of due or overdue screenings.

4. **Improve Patient Outcomes**  
   Increase adherence to preventive care protocols, leading to early detection and better disease management.

5. **Cost-Effective Implementation**  
   Leverage free and open-source tools to ensure accessibility and affordability for healthcare providers.

**Lets do some data processing**

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("data.csv")
df.head()

,Patient ID,Age,Gender,Chronic Disease,Preventive Measures,Quality of Life Score,Risk Factors,Lifestyle Interventions,BMI,Screening Status,...,Follow-Up Schedule,Risk Score,Priority Level,Health Advice,Diagnosis Date,Current Medications,Blood Pressure,Blood Sugar Levels,Doctor Name,Healthcare Provider
0,S0001,80,Female,Cancer Risk,Diet Plan,12.5,Smoking,Exercise,20.5,Completed,...,2/25/2026,70,Medium,Consider increasing physical activity.,2018-02-24,Atorvastatin,121/77,NaN,Catherine Thompson,ABC Health
1,S0002,36,Male,Heart Disease,Routine Screening,64.8,Sedentary Lifestyle,Yoga,27.1,Overdue,...,11/23/2026,4,Low,Consider quitting smoking.,2017-03-22,Atorvastatin,140/87,160.5,Katherine Strong,ABC Health
2,S0003,74,Male,Heart Disease,Smoking Cessation,57.1,Obesity,Exercise,31.2,Due,...,10/16/2024,97,High,Consider quitting smoking.,2023-11-25,Omeprazole,100/63,NaN,Joseph Collins,CarePlus
3,S0004,33,Male,Cancer Risk,Diet Plan,69.5,Smoking,Yoga,25.7,Overdue,...,8/13/2024,55,Medium,Consider quitting smoking.,2016-12-06,Metformin,98/66,78.4,Joshua Schmidt,CarePlus
4,S0005,67,Male,Heart Disease,Smoking Cessation,68.8,Family History,Yoga,25.8,Overdue,...,3/6/2026,46,Medium,Consider quitting smoking.,2020-06-09,Metformin,117/73,76.4,Ashlee Zimmerman,CarePlus


In [3]:
print("Column Names:")
print(df.columns)

Column Names:
Index(['Patient ID', 'Age', 'Gender', 'Chronic Disease', 'Preventive Measures',
       'Quality of Life Score', 'Risk Factors', 'Lifestyle Interventions',
       'BMI', 'Screening Status', 'Screening Completion Date',
       'Follow-Up Schedule', 'Risk Score', 'Priority Level', 'Health Advice',
       'Diagnosis Date', 'Current Medications', 'Blood Pressure',
       'Blood Sugar Levels', 'Doctor Name', 'Healthcare Provider'],
      dtype='object')


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Patient ID                 6000 non-null   object 
 1   Age                        6000 non-null   int64  
 2   Gender                     6000 non-null   object 
 3   Chronic Disease            6000 non-null   object 
 4   Preventive Measures        6000 non-null   object 
 5   Quality of Life Score      6000 non-null   float64
 6   Risk Factors               6000 non-null   object 
 7   Lifestyle Interventions    6000 non-null   object 
 8   BMI                        6000 non-null   float64
 9   Screening Status           6000 non-null   object 
 10  Screening Completion Date  3038 non-null   object 
 11  Follow-Up Schedule         6000 non-null   object 
 12  Risk Score                 6000 non-null   int64  
 13  Priority Level             6000 non-null   objec

In [5]:
df.shape

(6000, 21)

In [6]:
df.describe()

,Age,Quality of Life Score,BMI,Risk Score,Blood Sugar Levels
count,6000.000000,6000.000000,6000.000000,6000.000000,4794.000000
mean,49.025167,70.193533,25.011167,50.461000,134.520317
std,18.112322,14.877405,5.010807,28.903444,37.141933
min,18.000000,12.500000,5.400000,1.000000,70.000000
25%,33.000000,60.100000,21.700000,25.000000,102.600000
50%,49.000000,70.300000,25.100000,50.000000,134.400000
75%,65.000000,80.600000,28.400000,75.000000,166.300000
max,80.000000,130.300000,42.100000,100.000000,200.000000


**Handling missing values**

In [9]:
df['Blood Sugar Levels'] = df['Blood Sugar Levels'].fillna(df['Blood Sugar Levels'].mean())
df['Screening Completion Date'] = df['Screening Completion Date'].fillna('Not Available')

**Feature encoding for categorical variables**

In [15]:
categorical_cols = ['Gender', 'Chronic Disease', 'Preventive Measures', 'Risk Factors', 'Lifestyle Interventions', 'Screening Status', 'Priority Level', 'Health Advice', 'Current Medications', 'Doctor Name', 'Healthcare Provider']
encoder = OneHotEncoder(drop='first')

**Feature Scaling**

In [16]:
scaler = StandardScaler()
numerical_cols = ['Age', 'Quality of Life Score', 'BMI', 'Risk Score', 'Blood Sugar Levels']

**Preprocessing pipeline**

In [17]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler, numerical_cols),
        ('cat', encoder, categorical_cols)
    ])

**Split the dataset into features and target**

In [18]:
X = df.drop(['Patient ID', 'Diagnosis Date', 'Screening Completion Date'], axis=1)  # Exclude non-relevant columns
y = df['Risk Score']  # Assuming Risk Score is the target (can be adjusted)

**Train-test split**

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**XGBoost model pipeline**

In [21]:
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(use_label_encoder=False, eval_metric='logloss'))
])

**Hyperparameter tuning using GridSearchCV**

In [22]:
param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__learning_rate': [0.01, 0.1, 0.3],
    'classifier__max_depth': [3, 6, 10]
}

In [23]:
grid_search = GridSearchCV(xgb_pipeline, param_grid, cv=3, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


ValueError: 
All the 54 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
54 fits failed with the following error:
Traceback (most recent call last):
  File "D:\Anaconda\envs\data_analysis\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\Anaconda\envs\data_analysis\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Anaconda\envs\data_analysis\Lib\site-packages\sklearn\pipeline.py", line 473, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "D:\Anaconda\envs\data_analysis\Lib\site-packages\xgboost\core.py", line 726, in inner_f
    return func(**